In [1]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt

In [2]:
freq1 = 1200
freq2 = 2400

In [3]:
def openWavFile(filename):
    sampling_rate, data = wavfile.read(filename)
    print(f"sampling rate: {sampling_rate}")

    length = data.shape[0] / sampling_rate
    print(f"length = {length}s")
    
    return sampling_rate, data

In [4]:
def extract_peak_frequency(data, sampling_rate):
    fft_data = np.fft.fft(data)
    freqs = np.fft.fftfreq(len(data))
    
    peak_coefficient = np.argmax(np.abs(fft_data))
    peak_freq = freqs[peak_coefficient]
    
    return abs(peak_freq * sampling_rate)

In [5]:
def getChunks(milliseconds, sampling_rate):
    #divide this extracted data into 50ms chunks, and find the peak or dominant frequency of each chunk
    #now if the chunk's peak frequency is 1200Hz, then it is 0 bit and if it's 1400Hz, then it is 1 bit.
    #group these bits together and that's the received data
    try:
        milliseconds = milliseconds/1000

        delay = int(milliseconds*sampling_rate)
        print(f"delay between tones: {delay} samples")
        chunks = []

        data_length = len(data) - len(data)%delay
        prev = 0

        for i in range(delay, data_length, delay):
            chunks.append(data[prev:i])
            prev = i
    except:
        pass
    
    
    return chunks

In [6]:
def extractData(chunks, sampling_rate):
    try:
        peak_freqs = []
        for chunk in chunks:
            peak_freqs.append(round(extract_peak_frequency(chunk, sampling_rate), 0))
    except:
        pass
    
    return peak_freqs

In [7]:
def extractBits(peak_freqs):
    bits = []
    foundStartSequence = False
    foundEndSequence = False
    for frequency in peak_freqs:
        if(frequency >= 2495 and frequency <= 2505 and foundStartSequence == False):
            foundStartSequence = True
            print("Start sequence found!")
        
        elif(frequency >= 3495 and frequency <= 3505 and foundEndSequence == False):
            foundEndSequence = True
            print("End sequence found!")
            break
        
        elif(frequency >= freq1-10.0 and frequency <= freq1+10.0):
            bits.append(0)
        
        elif(frequency >= freq2-10.0 and frequency <= freq2+10.0):
            bits.append(1)
            
    bin_bits = []
    s = ""
    for i in range(len(bits)):
        if (i+1)%8 == 0:
            s += str(bits[i])
            bin_bits.append(s)
            s = ""
        else:
            s += str(bits[i])
            
    return bin_bits

In [8]:
def decodeAscii(bin_string):
    """binary_int = int(bin_string, 2);
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = "Bin string cannot be decoded"
    for enc in ['utf-8', 'ascii', 'ansi']:
        try:
            ascii_text = binary_array.decode(encoding=enc)
            break
        except:
            pass
    print(ascii_text)"""
    
    
    bin_to_str = "".join([chr(int(bin_string[i:i+8],2)) for i in range(0,len(bin_string),8)])

    return bin_to_str

In [9]:
filename = input("Enter the wave file with extension: ")
#now open the wav file
sampling_rate, data = openWavFile(filename)

#extract the chunks, each chunk is of 10ms in size
chunks = getChunks(10, sampling_rate)

Enter the wave file with extension: man.wav
sampling rate: 44100
length = 0.98s
delay between tones: 441 samples


In [10]:
#get the peak frequencies from the chunks
peak_freqs = extractData(chunks, sampling_rate)

In [11]:
bits = extractBits(peak_freqs)

In [12]:
bin_string = ""

for i in bits:
    bin_string += i
    
#print(bin_string)

#split the binary string at spaces, because, I don't know how to decode a space character(0100000)
bin_splits = bin_string.split("0100000")

In [13]:
for s in bin_splits:
    print(decodeAscii(s), end=" ")

hello there  